In [ ]:
grep ‘POST /’ access_log > post_today.txt

In [3]:
import sqlite3
import requests
import urllib.request
import json
import datetime

import os
os.system("rm Metries.db")

def metries_num():   
    #yesterday_f = open('log_yes.txt', 'r')
    users_yes = []
    #for line in yesterday_f: 
        #main_line_yes = line.strip().split()
        #if main_line_yes[7] == '200':
            #IP_yes = main_line_yes[0]
            #if IP_yes not in users_yes:
                #users_yes.append(IP_yes)

    today_f = open('log.txt', 'r')
    
    successful_request = 0
    users = []
    countries = []
    super_users = []
    sites = []
    hours = []
    hours_IP = []
    countries_IP = []
    countries_queries = []
    average_num = {}
    average_num_s = ''
    sites_from = []
    russian_texts = 0
    
    for line in today_f:
        main_line = line.strip().split()
        if main_line[8] == '200':
            IP = main_line[0]
            hour = main_line[3][12:15]
            site_in = main_line[6]
            site_from = main_line[10][1:-1]
            coun = json.loads((requests.get('http://api.sypexgeo.net/json/'+IP)).text)
            country = coun['country']['name_en']
            if country not in countries:
                countries.append(country)
                
            sites.append(site_in)
            hours.append(hour)
            countries_queries.append(country)
            successful_request += 1 #кол-во запросов
            
            if IP not in users:
                users.append(IP)
                hours_IP.append(hour)
                countries_IP.append(country)
                
            if site_from not in sites_from:
                sites_from.append(site_from)
            
            if 'en-rus' in line:
                russian_texts += 1 #кол-во переведенных страниц на русский
                
            if IP not in users_yes and IP in users:
                super_users.append(IP)
    
    names = [[sites, {}, 0], [hours, {}, 0], [hours_IP, {}, 0], [countries_IP, {}, ''], [countries_queries, {}, '']]
    
    for name in names:
        for i in name[0]:
            if i in name[1]:
                name[1][i] += 1
            else:
                name[1][i] = 1
             
    for k in range(0, 3):
        for i in (names[k][1]).values(): #Количество посещений самой часто посещаемой страницы
            if i > names[k][2]: #Час, когда было больше всего запросов к сайту
                names[k][2] = i #Час, когда было на сайте наибольшее число уникальных посетителей                
    
    for i in range(3, 5):
        for key in names[i][1].keys():       #кол-во уникальных пользователей по странам за день
            names[i][2] += key+'-'+str((names[i][1])[key])+',' #кол-во запросов по странам
     
    for key in names[3][1].keys():
        for key in names[4][1].keys():       #Среднее количество запросов на одного пользователя по странам
            average_num[key] = (names[4][1])[key] / (names[3][1])[key]  
    for key in average_num.keys():
        average_num_s += key+'-'+str(average_num[key])+','
        
    
    max_queries_site, max_queries_hour, max_IP_hour = names[0][2], names[1][2], names[2][2]
    countries_IP_s, countries_queries_s = names[3][2], names[4][2]
    super_unique_users_num = len(super_users) #кол-во пользователей, которые сегодня были на сайте, но не были вчера   
    unique_users_num = len(users) #кол-во уникальных пользователей
    countries_num = len(countries) #кол-во уникальных стран
    sites_from_num = len(sites_from) #кол-во уникальных сайтов, с которых пришли пользователи
    date = datetime.date.today()
    
    print(successful_request)
    print(max_queries_site)
    print(max_queries_hour)
    print(max_IP_hour)
    print(countries_IP_s)
    print(countries_queries_s)
    print(average_num_s)
    print(super_unique_users_num)
    print(unique_users_num)
    print(countries_num)
    print(sites_from_num)
    print(russian_texts)
    
    conn = sqlite3.connect('Metries.db')
    c = conn.cursor()
    
    c.execute("CREATE TABLE IF NOT EXISTS Metries_int (Date date, Succesful_requests INTEGER, Unique_users INTEGER, Countries INTEGER, Today_users INTEGER, Visiting_most_visited_page INTEGER,  Top_requests_hour INTEGER, Top_unique_users_hour INTEGER, Unique_outgoing_sites INTEGER, Russian_translatings INTEGER);")
    c.execute("CREATE TABLE IF NOT EXISTS Metries_countries (Date date, Unique_users_per_country text, Requests_per_country text, Average_num_of_requests_per_country text);")
    
    c.execute('''insert into Metries_int (Date, Succesful_requests, Unique_users, Countries, Today_users, Visiting_most_visited_page, Top_requests_hour, Top_unique_users_hour, Unique_outgoing_sites, Russian_translatings) 
        values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', (date, successful_request, unique_users_num, countries_num, super_unique_users_num, max_queries_site, max_queries_hour, max_IP_hour, sites_from_num, russian_texts))
    c.execute('''insert into Metries_countries (Date, Unique_users_per_country, Requests_per_country, Average_num_of_requests_per_country) 
        values (?, ?, ?, ?)''', (date, countries_IP_s, countries_queries_s, average_num_s))
   
    conn.commit()
    conn.close()
    
    
    
metries_num()

1
1
1
1
United States-1,
United States-1,
United States-1.0,
1
1
1
1
0
